In [1]:
import pandas as pd
import pvlib
from tqdm.notebook import tqdm
from pathlib import Path
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import os
from pynasapower.get_data import query_power
from pynasapower.geometry import point
import pandas as pd, datetime

In [2]:

def send_req(points):
    df = query_power(
        geometry     = points,
        start        = datetime.date(2010,1,1),
        end          = datetime.date(2025,1,1),
        community    = "re",
        parameters   = params,
        temporal_api = "daily",
        spatial_api  = "point",
        format       = "csv",
        to_file      = False,
    )
    return df
    
# params=["ALLSKY_SFC_SW_DWN","T2M","QV2M","PS","WS10M",
#       "CLRSKY_SFC_SW_DWN","ALLSKY_SFC_SW_DNI","ALLSKY_SFC_SW_DIFF"]
params=["ALLSKY_SFC_SW_DWN","CLRSKY_SFC_SW_DWN","ALLSKY_SFC_SW_DNI","ALLSKY_SFC_SW_DIFF",
        "ALLSKY_KT","T2M","RH2M","WS10M","CLOUD_AMT_DAY"]
            #lon , lat
cords={
    'karaj' : point(50.9950, 35.8400, "EPSG:4326"),
    'ardabil' : point(48.2933, 38.2498, "EPSG:4326"),
    'bushehr' : point(50.8203, 28.9234, "EPSG:4326"),
    'shahrekord' : point(50.8650, 32.3150, "EPSG:4326"),
    'tabriz' : point(46.2919, 38.0850, "EPSG:4326"),
    'shiraz' : point(52.5837, 29.5918, "EPSG:4326"),
    'rasht' : point(49.5832, 37.2808, "EPSG:4326"),
    'gorgan' : point(54.4478, 36.8431, "EPSG:4326"),
    'hamadan' : point(48.5150, 34.7981, "EPSG:4326"),
    'bandar_abbas' : point(56.2666, 27.1960, "EPSG:4326"),
    'ilam' : point(46.4226, 33.6369, "EPSG:4326"),
    'isfahan' : point(51.6675, 32.6546, "EPSG:4326"),
    'kerman' : point(57.0788, 30.2830, "EPSG:4326"),
    'kermanshah' : point(47.0650, 34.3142, "EPSG:4326"),
    'ahvaz' : point(48.6706, 31.3183, "EPSG:4326"),
    'yasuj' : point(51.5875, 30.6684, "EPSG:4326"),
    'sanandaj' : point(47.0028, 35.3098, "EPSG:4326"),
    'khorramabad' : point(48.3558, 33.4878, "EPSG:4326"),
    'arak' : point(49.6892, 34.0917, "EPSG:4326"),
    'sari' : point(53.0601, 36.5633, "EPSG:4326"),
    'bojnurd' : point(57.3290, 37.4742, "EPSG:4326"),
    'qazvin' : point(50.0041, 36.2688, "EPSG:4326"),
    'qom' : point(50.8764, 34.6400, "EPSG:4326"),
    'mashhad' : point(59.6168, 36.2605, "EPSG:4326"),
    'semnan' : point(53.3978, 35.5727, "EPSG:4326"),
    'zahedan' : point(60.8629, 29.4963, "EPSG:4326"),
    'birjand' : point(59.2211, 32.8663, "EPSG:4326"),
    'tehran' : point(51.3890, 35.6892, "EPSG:4326"),
    'urmia' : point(45.0728, 37.5524, "EPSG:4326"),
    'yazd' : point(54.3569, 31.8974, "EPSG:4326"),
    'zanjan' : point(48.4787, 36.6736, "EPSG:4326")
}

output_dir="../Dataset/Nasa Datasets/"

In [3]:
# for city in cords:
#     print(city)
#     cord=cords[city]
#     df=send_req(cord) 
#     df.to_csv(os.path.join(output_dir,f"{city}.csv"),index=False)

In [4]:
# def to_week(day):
#     if(day<=7):
#         return 1
#     if(day>7 and day<=14):
#         return 2
#     if(day>14 and day<=21):
#         return 3
#     if(day>21 and day<=28):
#         return 4
#     if(day>28):
#         return 5
# for city in cords:
#     df=pd.read_csv(os.path.join(output_dir,f"{city}.csv"))
#     df["week"]=df["DY"].apply(to_week)
#     df.to_csv(os.path.join(output_dir,f"{city}.csv"),index=False)
# df=pd.read_csv(os.path.join(output_dir,"kerman.csv"))

In [5]:
def calculate_features(df,week,month):
    d=df[(df["week"]==week) & (df["MO"]==month)]
    year=df["YEAR"].max()
    print(f"predicted features for the year : {year+1} \n month : {month} \n week : {week}")
    display(d.drop(columns=["week","MO","YEAR","DY"]).mean())
week=2
month=3
# calculate_features(df,week,month)

In [27]:
selected_pannel="aSiMicro03036"
modual_infos=pd.read_csv("./simulation prams/module_params_custom_material.csv",index_col="Model")
# temp_infos =pd.read_csv("./simulation prams/Temperature_Parameters_for_Each_PV_Model.csv",index_col="Model",usecols=["Model","a","b","deltaT"])
# cec_inverter = pvlib.pvsystem.retrieve_sam(name="CECInverter") 

# inverter_params =cec_inverter["ABB__PVI_3_0_OUTD_S_US__208V_"]
module_params = modual_infos.loc[selected_pannel]
# display(module_params)
pdc0=module_params.loc["Parallel_Strings"]*module_params.loc["Isco"]*module_params.loc["Voco"]
# temp_params=temp_infos.loc[selected_pannel]
module_params = {
    'gamma_pdc': -0.0038,
    'pdc0':pdc0
}
# module_params = {
#     'pdc0':      700, 
#     'gamma_pdc': -0.0038,
# }
inverter_params = {
    'pdc0': pdc0,
    'eta_inv_nom': 0.96
}

In [41]:
for city in tqdm(cords):
    df=pd.read_csv(os.path.join(output_dir,f"{city}.csv"))
    input_df=pd.DataFrame()
    input_df["Date"] = pd.to_datetime({
        'year':  df['YEAR'],
        'month': df['MO'],
        'day':   df['DY']
    })
    input_df["Date"]=input_df["Date"].dt.tz_localize('Asia/Tehran',nonexistent='shift_forward')
    input_df['ghi_Wm2'] = df['ALLSKY_SFC_SW_DWN'] * 1000.0 /24
    input_df['temp_air']   = df['T2M']
    input_df['wind_speed'] = df['WS10M']
    input_df["T2M"]=df["T2M"]
    input_df.set_index("Date",drop=True,inplace=True)
    input_df = input_df.resample('h').ffill()
    # display(input_df)
    dc_power = pvlib.pvsystem.pvwatts_dc(
        effective_irradiance=input_df['ghi_Wm2'],
        temp_cell=input_df["T2M"],
        temp_ref=input_df['temp_air'],
        **module_params
    )
    ac_power = pvlib.inverter.pvwatts(
        pdc=dc_power,
        pdc0=inverter_params['pdc0'],
        eta_inv_nom=inverter_params['eta_inv_nom']
    )
    input_df["P"]=ac_power.values
    df["P (W)"]=input_df.resample('D').sum()["P"].values
    # df['P (W)'] = ac_power *24
    # df.to_csv(os.path.join(output_dir,f"{city}.csv"),index=False)
    print(f"{city} : ",df["P (W)"].mean())
    

  0%|          | 0/31 [00:00<?, ?it/s]

karaj :  926.1914350372288
ardabil :  740.1366565333409
bushehr :  1001.6902912822868
shahrekord :  976.5618609418782
tabriz :  833.4744746265841
shiraz :  1043.8671219282885
rasht :  685.0127727676022
gorgan :  852.1336539733356
hamadan :  949.5353008637254
bandar_abbas :  1022.8616359804583
ilam :  968.97846672588
isfahan :  1016.9045080802665
kerman :  1038.1484163009984
kermanshah :  949.2096690430673
ahvaz :  981.9230724031092
yasuj :  1014.306915843203
sanandaj :  949.3276442029609
khorramabad :  958.7280372243267
arak :  948.7938940986637
sari :  773.6868616191827
bojnurd :  877.3442282811322
qazvin :  797.8346842490422
qom :  943.4098311694373
mashhad :  903.2299305143637
semnan :  921.5630414717657
zahedan :  1059.0040739553544
birjand :  1015.3237228805253
tehran :  917.5002084024715
urmia :  883.3131243527921
yazd :  1033.4976422328436
zanjan :  905.6061044831771
